implementing a very naive attention mechanism 

In [3]:
import torch 

inputs = torch.tensor(
[[0.43, 0.15, 0.89, 0.43, 0.15, 0.89], 
[0.55, 0.87, 0.66, 0.55, 0.87, 0.66], 
[0.57, 0.85, 0.64, 0.57, 0.85, 0.64],  
[0.22, 0.58, 0.33, 0.57, 0.85, 0.64],  
[0.77, 0.25, 0.10, 0.57, 0.85, 0.64],  
[0.05, 0.80, 0.55, 0.57, 0.85, 0.64]])

In [4]:
attn_scores = torch.zeros(inputs.shape[0], inputs.shape[0])

'''for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i][j] = torch.dot(x_i, x_j)'''


attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[1.9990, 1.9088, 1.8844, 1.4175, 1.3998, 1.5732],
        [1.9088, 2.9900, 2.9508, 2.3188, 2.1824, 2.5619],
        [1.8844, 2.9508, 2.9140, 2.2866, 2.1724, 2.5175],
        [1.4175, 2.3188, 2.2866, 1.9507, 1.8044, 2.1135],
        [1.3998, 2.1824, 2.1724, 1.8044, 2.1224, 1.7505],
        [1.5732, 2.5619, 2.5175, 2.1135, 1.7505, 2.4020]])


In [5]:
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights


tensor([[0.2190, 0.2001, 0.1953, 0.1224, 0.1203, 0.1430],
        [0.0868, 0.2558, 0.2460, 0.1307, 0.1141, 0.1667],
        [0.0876, 0.2544, 0.2452, 0.1310, 0.1168, 0.1650],
        [0.0907, 0.2233, 0.2162, 0.1545, 0.1335, 0.1818],
        [0.0968, 0.2117, 0.2096, 0.1451, 0.1994, 0.1375],
        [0.0873, 0.2345, 0.2244, 0.1498, 0.1042, 0.1999]])

In [6]:
#context_vec = attn_weights @


context_vec = torch.zeros(inputs.shape)


'''for i, a_i in enumerate(attn_weights):
    for j, x_i in enumerate(inputs):
        context_vec[i] += a_i[j] * x_i'''

context_vec = attn_weights @ inputs
context_vec


tensor([[0.4422, 0.5884, 0.5830, 0.5353, 0.7007, 0.6987],
        [0.4431, 0.6823, 0.5497, 0.5527, 0.7944, 0.6668],
        [0.4444, 0.6801, 0.5485, 0.5526, 0.7938, 0.6670],
        [0.4309, 0.6601, 0.5308, 0.5528, 0.7910, 0.6671],
        [0.4698, 0.6208, 0.5034, 0.5522, 0.7865, 0.6684],
        [0.4176, 0.6807, 0.5458, 0.5531, 0.7936, 0.6665]])

coding a better, trainable attention mechanism

In [7]:
x_2 = inputs[1]
d_in = x_2.shape[0]
d_out = x_2.shape[0] - 1

In [8]:
torch.manual_seed(123)

w_q = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # query matrix
w_k = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # key matrix
w_v = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # value matrix 

In [9]:
x_q = x_2 @ w_q 
x_v = x_2 @ w_v
x_k = x_2 @ w_k
keys = inputs @ w_k # keys for all the input elements
values = inputs @ w_v # values for all the input elements

In [10]:
print(x_k)
print(x_q)
print(x_v)

tensor([2.0521, 1.5140, 2.4725, 2.6323, 2.1560])
tensor([2.5229, 1.6209, 1.8523, 2.3272, 1.5094])
tensor([2.5138, 2.3351, 1.6296, 1.3288, 2.3814])


In [11]:
print(keys)
print(values)

tensor([[1.1804, 0.9781, 1.9330, 2.1744, 1.3799],
        [2.0521, 1.5140, 2.4725, 2.6323, 2.1560],
        [2.0499, 1.5150, 2.4513, 2.6184, 2.1599],
        [1.4823, 1.1638, 1.9629, 2.0296, 1.5527],
        [1.7553, 1.4913, 2.1355, 2.1451, 1.7745],
        [1.5097, 1.1062, 2.0193, 2.1564, 1.6157]])
tensor([[1.7690, 1.7220, 1.1177, 1.1184, 1.8703],
        [2.5138, 2.3351, 1.6296, 1.3288, 2.3814],
        [2.4855, 2.3294, 1.5882, 1.3147, 2.3576],
        [1.8515, 1.6566, 1.4132, 1.2241, 1.8347],
        [1.7823, 1.8470, 1.2479, 1.2875, 1.5952],
        [2.0740, 1.7629, 1.5386, 1.2204, 2.1008]])


In [12]:
attn_scores_2 = x_q @ keys.T # dot product of (x^2)_q with (x^i)_k for all ^ as index in the inputs 
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)

In [13]:
attn_weights_2

tensor([0.0240, 0.4017, 0.3894, 0.0382, 0.0996, 0.0471])

In [14]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([2.3661, 2.2168, 1.5507, 1.3050, 2.2475])

In [15]:
import torch.nn as nn
'''implementing a python class for the above, improved attention'''

class SelfAttentionV1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        keys = x @ self.W_key
        values = x @ self.W_value
        queries = x @ self.W_query

        attention_weights = torch.softmax((queries @ keys.T) / keys.shape[-1] ** 0.5, dim=-1)
        context_vec = attention_weights @ values
        return context_vec


In [16]:
torch.manual_seed(123)
sa_v1 = SelfAttentionV1(d_in, d_out)
print(sa_v1(inputs))

tensor([[2.2807, 2.1373, 1.5133, 1.2905, 2.1767],
        [2.3661, 2.2168, 1.5507, 1.3050, 2.2475],
        [2.3639, 2.2148, 1.5497, 1.3047, 2.2457],
        [2.3280, 2.1811, 1.5342, 1.2986, 2.2160],
        [2.3255, 2.1786, 1.5329, 1.2979, 2.2144],
        [2.3397, 2.1922, 1.5394, 1.3008, 2.2253]], grad_fn=<MmBackward0>)


In [17]:
import torch.nn as nn
'''re-implementing the class with nn.Linear for better weight initialization '''

class SelfAttentionV2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)

        attention_weights = torch.softmax((queries @ keys.T) / keys.shape[-1] ** 0.5, dim=-1)
        context_vec = attention_weights @ values
        return context_vec


In [18]:
torch.manual_seed(123)
sa_v2 = SelfAttentionV2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.2370,  0.2580, -0.3748,  0.3019,  0.1255],
        [-0.2384,  0.2577, -0.3741,  0.2964,  0.1261],
        [-0.2384,  0.2578, -0.3740,  0.2963,  0.1262],
        [-0.2375,  0.2602, -0.3735,  0.2969,  0.1279],
        [-0.2381,  0.2593, -0.3730,  0.2963,  0.1277],
        [-0.2375,  0.2599, -0.3739,  0.2970,  0.1275]], grad_fn=<MmBackward0>)


In [19]:
sa_v2 = SelfAttentionV2(d_in, d_out)
wk = sa_v2.W_key.weight.T
wq = sa_v2.W_query.weight.T

In [20]:
keys = inputs @ wk
queries = inputs @ wq

In [21]:
attn_scores = queries @ keys.T
type(attn_scores.shape)

torch.Size

In [22]:
mask = torch.triu(torch.ones(attn_scores.shape[0], attn_scores.shape[1]), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
attn_weight = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=1)
dropout = torch.nn.Dropout(0.3)
dropout(attn_weight)


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4771, 0.4770, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3584, 0.0000, 0.3518, 0.0000, 0.0000],
        [0.2948, 0.2793, 0.0000, 0.2806, 0.2936, 0.0000],
        [0.2378, 0.2435, 0.2432, 0.0000, 0.2312, 0.2385]],
       grad_fn=<MulBackward0>)

In [23]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias = False):
        super().__init__()
        self.W_K = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_Q = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_V = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        keys = self.W_K(x)
        values = self.W_V(x)
        queries = self.W_Q(x)
        attn_scores = queries @ keys.transpose(1,2)
        attn_scores = attn_scores.masked_fill(self.mask.bool(), -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values

        return context_vec


In [24]:
batch = torch.stack((inputs, inputs))
context_length = batch.shape[1]

ca = CausalAttention(d_in, d_out, context_length, 0.0)
cv = ca(batch)
print(cv)

tensor([[[-0.3642,  0.0227,  0.2398, -0.0169,  0.4638],
         [-0.2611,  0.0933,  0.0065, -0.0643,  0.2690],
         [-0.2280,  0.1148, -0.0628, -0.0737,  0.2039],
         [-0.2038,  0.1369, -0.1097, -0.0577,  0.1818],
         [-0.2348,  0.1917, -0.0948, -0.0141,  0.1385],
         [-0.2006,  0.1720, -0.1355, -0.0328,  0.1483]],

        [[-0.3642,  0.0227,  0.2398, -0.0169,  0.4638],
         [-0.2611,  0.0933,  0.0065, -0.0643,  0.2690],
         [-0.2280,  0.1148, -0.0628, -0.0737,  0.2039],
         [-0.2038,  0.1369, -0.1097, -0.0577,  0.1818],
         [-0.2348,  0.1917, -0.0948, -0.0141,  0.1385],
         [-0.2006,  0.1720, -0.1355, -0.0328,  0.1483]]],
       grad_fn=<UnsafeViewBackward0>)


internally, pytorch flattens the tensors (B, T, D) -> (B*T, D), does the operations and then reconstructs the original dims in case of batch operations.

### Why one head is not enough

- Limited subspace: A single attention head attends in a fixed representation space. It can't simultaneously focus on diverse semantic/structural patterns (like subject-object relationships and polysemy resolution).

- Capacity bottleneck: One head has fewer parameters and thus limited capacity to model all the nuances. Multi-head splits the job into specialized roles — one does syntax, another semantics, etc.

- Inductive bias: Multi-head attention acts like an ensemble. Each head learns a different perspective. This enforces modularity and improves generalization.

- Empirical evidence: Multi-head models like Transformers perform significantly better than single-head counterparts on benchmarks (e.g., BLEU for translation, GLUE for NLP tasks).

In [25]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))


    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)       # splitting the matrix in order to
        values = values.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)   # generate (B, NUM_HEADS) independent matrices
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2) # of size (NUM_TOKENS, HEAD_DIM) for parallel
                                                                                             # computation (hence, multi-head attention)

        attn_scores = queries @ keys.transpose(2, 3)
        mask = torch.triu(torch.ones(num_tokens, num_tokens, device=x.device), diagonal=1).bool()
        attn_scores = attn_scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), float('-inf'))
        print(attn_scores.shape)
        print(attn_scores)
        

        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vector = (attn_weights @ values).transpose(1, 2)
        context_vector = context_vector.contiguous().view(b, num_tokens, self.d_out)
        context_vector = self.out_proj(context_vector)

        return context_vector

        

        



In [26]:
torch.manual_seed(123)
batch = torch.stack((inputs, inputs, inputs, inputs, inputs, inputs, inputs, inputs), dim=0)
batch_size, context_length, d_in = batch.shape
d_out = 2

mha = MultiHeadAttention(d_in=6, d_out=6, context_length=6, dropout=0, num_heads=3)
cv = mha(batch)
print(cv)


torch.Size([8, 3, 6, 6])
tensor([[[[-0.0501,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-0.0193, -0.1214,    -inf,    -inf,    -inf,    -inf],
          [-0.0191, -0.1195, -0.1144,    -inf,    -inf,    -inf],
          [-0.0098, -0.0593, -0.0568, -0.0385,    -inf,    -inf],
          [ 0.0080, -0.0360, -0.0336, -0.0225,  0.0316,    -inf],
          [-0.0195, -0.0856, -0.0823, -0.0559,  0.0072, -0.0949]],

         [[ 0.1333,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.1528,  0.1335,    -inf,    -inf,    -inf,    -inf],
          [ 0.1490,  0.1287,  0.1217,    -inf,    -inf,    -inf],
          [ 0.1227,  0.1178,  0.1123,  0.0234,    -inf,    -inf],
          [ 0.0925,  0.0716,  0.0672, -0.0339, -0.1419,    -inf],
          [ 0.1455,  0.1444,  0.1379,  0.0418, -0.1499,  0.1617]],

         [[-0.0107,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.1747,  0.0356,    -inf,    -inf,    -inf,    -inf],
          [ 0.1752,  0.0386,  0.0356,    -inf, 

In [27]:
batch.shape


torch.Size([8, 6, 6])

In [28]:
total_params = sum(a.numel() for a in mha.parameters())
total_params

150

#### The first n-2 dims define how many independent matrix multiplications happen in parallel.


Imagine you have a batch of data or multiple sequences you want to process simultaneously:

Those leading dims act as batch dimensions or sequence dims.

PyTorch treats each slice along those dims as an independent matrix.

It runs the matrix multiplication on all these slices in parallel efficiently on GPU/CPU.

#### Multi-Head Attention Example

Input Sentence: "I go to the bank to deposit money"

Each attention head processes the same input, but focuses on different relationships due to different learned projections (W_q, W_k, W_v).

##### Head 1: Semantic Disambiguation
- Focus: "bank" ↔ "deposit", "money"
- Learns that "bank" is a financial institution, not a river bank.

##### Head 2: Subject Identification
- Focus: "I" ↔ "go", "to"
- Captures who is acting — the grammatical subject.

##### Head 3: Grammatical Structure
- Focus: "to" ↔ "go", "deposit"
- Models motion or action phrases — verb-preposition-object relationships.


Multi-head attention enables different heads to learn specialized roles by attending in different representational subspaces, improving model expressiveness.